In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
# load runners data
runners_df = pd.read_csv('MyBerlin_1999-2023.csv')

# load weather data
weather_df = pd.read_csv('Berlin_Marathon_weather_data_since_1974.csv')

# print data preview
print(runners_df.head())
print(weather_df.head())

   index  Year  platz  nachname   vorname nation  ak sex        z5       z10  \
0      0  1999      1   Kiprono  Josephat    KEN  25   M  00:15:09  00:30:21   
1      1  1999      2  Inubushi  Takayuki    JPN  25   M  00:15:09  00:30:21   
2      2  1999      3    Kandie    Samson    KEN  25   M  00:15:10  00:30:21   
3      3  1999      4     Chatt    Hicham    MAR  25   M  00:15:11  00:30:23   
4      4  1999      5   Cherono     Henry    KEN  25   M  00:15:10  00:30:21   

        z15       z20       z25       z30       z35       z40 halbmarathon  \
0  00:45:25  01:00:34  01:14:54  01:30:10  01:45:05  02:00:08     01:03:54   
1  00:45:27  01:00:36  01:14:56  01:30:18  01:45:21  02:00:18     01:03:54   
2  00:45:27  01:00:35  01:14:55  01:30:18  01:45:59  02:01:41     01:03:54   
3  00:45:27  01:00:36  01:14:56  01:30:24  01:46:14  02:02:52     01:03:55   
4  00:00:00  01:00:35  01:14:55  01:30:18  01:46:14  02:02:58     01:03:55   

      netto  
0  02:06:44  
1  02:06:57  
2  02:08

d:\IFT\data visualization\tp1\exercices_python\venv\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
weather_df = weather_df.rename(columns={'YEAR': 'Year'})
# merge two dataframes
merged_df = pd.merge(runners_df, weather_df, on='Year', how='left')

# print merged data preview
print(merged_df.head())

   index  Year  platz  nachname   vorname nation  ak sex        z5       z10  \
0      0  1999      1   Kiprono  Josephat    KEN  25   M  00:15:09  00:30:21   
1      1  1999      2  Inubushi  Takayuki    JPN  25   M  00:15:09  00:30:21   
2      2  1999      3    Kandie    Samson    KEN  25   M  00:15:10  00:30:21   
3      3  1999      4     Chatt    Hicham    MAR  25   M  00:15:11  00:30:23   
4      4  1999      5   Cherono     Henry    KEN  25   M  00:15:10  00:30:21   

   ...       z40 halbmarathon     netto PRECIP_mm SUNSHINE_hrs CLOUD_hrs  \
0  ...  02:00:08     01:03:54  02:06:44       1.2          0.0       6.3   
1  ...  02:00:18     01:03:54  02:06:57       1.2          0.0       6.3   
2  ...  02:01:41     01:03:54  02:08:31       1.2          0.0       6.3   
3  ...  02:02:52     01:03:55  02:09:56       1.2          0.0       6.3   
4  ...  02:02:58     01:03:55  02:10:37       1.2          0.0       6.3   

  ATMOS_PRESS_mbar AVG_TEMP_C  MAX_TEMP_C  MIN_TEMP_C  
0     

In [6]:
# convert time to minutes
def time_to_minutes(time_str):
    h, m, s = map(int, time_str.split(':'))
    return h * 60 + m + s / 60

merged_df['netto_minutes'] = merged_df['netto'].apply(time_to_minutes)

# print converted time
print(merged_df[['netto', 'netto_minutes']].head())

      netto  netto_minutes
0  02:06:44     126.733333
1  02:06:57     126.950000
2  02:08:31     128.516667
3  02:09:56     129.933333
4  02:10:37     130.616667


In [7]:
# mark elite runners
merged_df['is_elite'] = merged_df.groupby('Year')['platz'].transform(lambda x: x <= 10)

print(merged_df[['Year', 'platz', 'is_elite']].head(10))

   Year  platz  is_elite
0  1999      1      True
1  1999      2      True
2  1999      3      True
3  1999      4      True
4  1999      5      True
5  1999      6      True
6  1999      7      True
7  1999      8      True
8  1999      9      True
9  1999     10      True


In [ ]:
def assign_level(series):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    conditions = [
        (series <= q1), # 0-25%, high
        (series > q1) & (series <= q3), # 25-75%, medium
        (series > q3) # 75-100%, low
    ]
    choices = ['High', 'Medium', 'Low']
    return np.select(conditions, choices, default='Medium')

# assign level
merged_df['level'] = merged_df.groupby('Year')['netto_minutes'].transform(assign_level)

print(merged_df.head(20))

    index  Year  platz         nachname     vorname nation  ak sex        z5  \
0       0  1999      1          Kiprono    Josephat    KEN  25   M  00:15:09   
1       1  1999      2         Inubushi    Takayuki    JPN  25   M  00:15:09   
2       2  1999      3           Kandie      Samson    KEN  25   M  00:15:10   
3       3  1999      4            Chatt      Hicham    MAR  25   M  00:15:11   
4       4  1999      5          Cherono       Henry    KEN  25   M  00:15:10   
5       5  1999      6           Glinka    Waldemar    POL  30   M  00:15:11   
6       6  1999      7       Nascimento  Eduardo do    BRA  25   M  00:15:11   
7       7  1999      8           Ndungu     Stephen    KEN  30   M  00:15:11   
8       8  1999      9         Tokunaga     Daisuke    JPN  30   M  00:15:13   
9       9  1999     10         de Souza  José Teles    BRA  25   M  00:00:00   
10     10  1999     11          Kariuki      Joseph    KEN  25   M  00:15:11   
11     11  1999     12           Thlobo 